# Wave equation on a disk

Juan Llanos

Physics 6810

Ralf Bundschuh

04/24/20

In this notebook, we'll solve the wave equation for $u$,

$\begin{align}
  \frac{\partial^2 u}{\partial t^2} = c^2\nabla^2u
\end{align}$

by a finite difference method, given its initial shape and velocity at $t=0$ on a circular domain. Since the domain is a circle, we'll write the Laplacian $\nabla^2$ in polar coordinates

$\begin{align}
  \nabla^2u=\frac{\partial^2u}{\partial r^2}+\frac{1}{r}\frac{\partial u}{\partial r}
  +\frac{1}{r^2}\frac{\partial u}{\partial\theta^2}
\end{align}$

Thus our PDE becomes

$\begin{align}
\frac{\partial^2 u(x,y,t)}{\partial t^2} = c^2\left(\frac{\partial^2u}{\partial r^2}
+\frac{1}{r}\frac{\partial u}{\partial r}
+\frac{1}{r^2}\frac{\partial u}{\partial\theta^2}\right)
\end{align}$

With $0\leq r\leq a$, $0\leq \theta \leq 2\pi$, $a=1$, and $c=1$

For simplicity, we'll begin with an initial shape that is an eigenmode. If we expand $u(r,\theta, t)$ in terms of the eigenmodes we get

$\begin{align*}
u(r,\theta,t) = &\sum_{n=1}^{\infty}\sum_{m=1}^{\infty}J_n(\sqrt{\lambda_{nm}} r)\cos(ct\sqrt{\lambda_{nm}})
\left(A_{nm}\cos(n\theta)+B_{nm}\sin(n\theta)\right)\\
+ &\sum_{n=1}^{\infty}\sum_{m=1}^{\infty}J_n(\sqrt{\lambda_{nm}} r)\sin(ct\sqrt{\lambda_{nm}})
\left(C_{nm}\cos(n\theta)+D_{nm}\sin(n\theta)\right)
\end{align*}$

Where $J_n(x)$ is the Bessel function of the first kind of order $n$, $\sqrt{\lambda_{nm}}$ is the ratio $z_{nm}/a$,and $z_{nm}$ is the m-th zero of the Bessel function of order $n$. We'll pick the mode that corresponds to $A_{3,1}=1$ and all other $A_{nm}$, $B_{nm}$, $C_{nm}$, and $D_{nm}$ are all zero. Thus our initial shape is


$\begin{align}
  u(r, \theta,0) = J_3(z_{3,1}r)\sin(3\theta)\qquad \text{with $a=1$ and $c=1$}
\end{align}$

and the velocity is zero for all $r$ & $\theta$ at $t=0$. Since the eigenmodes evolve independently in time, we expect our solution to be

$\begin{align}
  u(r,\theta,t)=J_3(z_{3,1}r)\sin(3\theta)\cos(tz_{3,1}) \qquad \text{with $a=1$ and $c=1$}
\end{align}$

We'll use the scipy special functions library throught this code.

This code was adapted from another code provided by Dr. Dick Furnstahl for a homework exercise for physics 5300
* Created 03-Apr-2019. Last revised 06-Apr-2019 by Dick Furnstahl (furnstahl.1@osu.edu).
* Modified 23-Apr-2020 by Juan Llanos (llanos.5@osu.edu).
* Modified 24-Apr-2020 by Juan Llanos (llanos.5@osu.edu).


# Setting up finite diference equation

To do this problem in terms of finite differences, we need discretized versions of partial derivatives. We start with taking a Taylor series of $u$ and expanding about $r\pm\Delta r$

$\begin{align*}
u(r+\Delta r,\theta,t)=&u(r, \theta, t)+\frac{\partial u}{\partial r}(\Delta r)
+\frac{1}{2}\frac{\partial^2 u}{\partial r^2}(\Delta r)^2
+\frac{1}{6}\frac{\partial^3 u}{\partial r^3}(\Delta r)+\mathcal{O}(\Delta r)^4\\
u(r-\Delta r,\theta,t)=&u(r, \theta, t)-\frac{\partial u}{\partial r}(\Delta r)
+\frac{1}{2}\frac{\partial^2 u}{\partial r^2}(\Delta r)^2
-\frac{1}{6}\frac{\partial^3 u}{\partial r^3}(\Delta r)+\mathcal{O}(\Delta r)^4\\
\end{align*}$

Next, we'll add the two equations and solve for the second partial derivative.

$\begin{align}
\frac{\partial^2 u}{\partial r^2}=\frac{u(r+\Delta r,\theta,t)-2u(r, \theta, t)+u(r-\Delta r,\theta,t)}{(\Delta r)^2}
+\mathcal{O}(\Delta r)^3
\end{align}$

If we subtract these two equations, we also get the central difference formula

$\begin{align}
\frac{\partial u}{\partial r}=\frac{u(r+\Delta r,\theta,t)-u(r-\Delta r,\theta,t)}{\Delta r}
+\mathcal{O}(\Delta r)^2
\end{align}$

We repeat the procedure for the  second partial derivative with respect to $\theta$

$\begin{align}
\frac{\partial^2 u}{\partial \theta^2}=\frac{u(r,\theta+\Delta \theta,t)-2u(r, \theta, t)+u(r,\theta-\Delta \theta,t)}{(\Delta \theta)^2}
+\mathcal{O}(\Delta \theta)^3
\end{align}$

We'll do this for the second partial derivative with respect to $t$, but we'll solve for $u(r,\theta,t+\Delta t)$ which will give us a method for evaluating our function at future time values

$\begin{align}
u(r,\theta, t+\Delta t) \approx 2u(r,\theta,t)-u(r,\theta,t-\Delta t)+\frac{\partial^2 u}{\partial t^2}(\Delta t)^2
\end{align}$

We can use the wave equation to replace the second partial with respect to time with $c\nabla^2 u$ which in finite difference form is

$\begin{align}
\nabla^2u\approx \frac{u(r+\Delta r,\theta,t)-2u(r, \theta, t)+u(r-\Delta r,\theta,t)}{(\Delta r)^2}
+\frac{1}{r}\frac{u(r+\Delta r,\theta,t)-u(r-\Delta r,\theta,t)}{\Delta r}
+\frac{1}{r^2}\frac{u(r,\theta+\Delta \theta,t)-2u(r, \theta, t)+u(r,\theta-\Delta \theta,t)}{(\Delta \theta)^2}
\end{align}$

Putting this all together gets us

$\begin{align*}
u(r,\theta, t+\Delta t) \approx& 2u(r,\theta,t)-u(r,\theta,t-\Delta t)\\
+&c^2(\Delta t)^2\left(\frac{u(r+\Delta r,\theta,t)-2u(r, \theta, t)+u(r-\Delta r,\theta,t)}{(\Delta r)^2}\right)\\
+&c^2(\Delta t)^2\left(\frac{1}{r}\frac{u(r+\Delta r,\theta,t)-u(r-\Delta r,\theta,t)}{\Delta r}\right)\\
+&c^2(\Delta t)^2\left(\frac{1}{r^2}
\frac{u(r,\theta+\Delta \theta,t)-2u(r, \theta, t)+u(r,\theta-\Delta \theta,t)}{(\Delta \theta)^2}\right)
\end{align*}$

This equation requires that we know $u$ at time values $t$ and $t-\Delta t$ to get $u$ at $t+\Delta t$. At $t=0$, we need $u(r, \theta, -\Delta t)$ to determine $u(r,\theta,\Delta t)$ . To work around this, we use the backward difference formula and solve to $u(r, \theta, -\Delta t)$

$\begin{align}
u(r, \theta, -\Delta t) \approx u(r, \theta, 0) -\frac{\partial u(r,\theta,0)}{\partial r}(\Delta t)
\end{align}$

Now we have a way of determining $u$ in the future, in the present, and in the past

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as sp
# This import registers the 3D projection, but is otherwise unused.
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401 unused import

from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter

from matplotlib import animation, rc
from IPython.display import HTML

In [15]:
class uDisk():
    """
    uDisk class sets up a wave at x_pts.  Now with finite difference.
     
    Parameters
    ----------
    r_pts : array of floats
        r values of the wave
    theta_pts : array of floats
        theta values of the wave 
    delta_t : float
        time step
    c : float
        speed of the wave
    a : float
        Radius of the disk
    n : float
        order of Bessel function
    m : float
        m-th zero of the n-th order Bessel function

    Methods
    -------
    u_wave(self, t)
        Returns u(x, t) for x in x_pts and time t 
    """
    def __init__(self, r_pts, theta_pts, delta_t=0.01, c_wave=1., a=1., n=3, m=1):
        
        self.r_pts = r_pts
        self.theta_pts = theta_pts
        self.a = a
        self.delta_t = delta_t
        self.c = c_wave
        self.delta_r = r_pts[1] - r_pts[0]
        self.delta_theta = theta_pts[1]-theta_pts[0]
        self.n = n
        self.m = m
        
        self.u_start()  # set the starting functions
    def J_n(self,r):
        """Defines J_n(z_{nm}/a*r)"""
        zeros = sp.jn_zeros(self.n,self.m)
        #jn_zeros returns the first m zeros of J_n. We only want the last element of this list
        z = zeros[-1]
        return sp.jv(self.n,z/self.a*r)

    def u_0(self, r, theta):
        """Initial shape of amplitude on the disk."""
        return self.J_n(r)*np.sin(self.n*theta)
    
    
    def u_dot_0(self,r, theta):
        """Initial velocity of amplitude on the disk."""
        return 0.
    
    def u_start(self):
        """Initiate u_past and u_present."""
        # u_present & u_dot_present are arrays where each array element is u (the solution to our pde) evaluated at
        # each node r_i, theta_j which we can denote as u(r_i,theta_j,0) =u_{ij}(0)
        # This function is supposed to iterate through each theta node at fixed radius and then increase the radius 
        # by delta_r and repeat the theta iterations until it reaches the boundary of the disk
        
        self.u_present = np.zeros((len(r_pts),len(theta_pts)))
        self.u_dot_present = np.zeros((len(r_pts),len(theta_pts)))
        self.u_past = np.zeros((len(r_pts),len(theta_pts)))
        # Note: changed the arange starting values from 1 to 0 since starting with 1 it didn't evaluate u(r=0,theta)
        # may need to adjust indexing 
        # 
        for i in np.arange(0, len(r_pts) - 1):
            r = self.r_pts[i]
            for j in np.arange(0, len(theta_pts)-1):
                theta = self.theta_pts[j]
                self.u_present[i,j] = self.u_0(r,theta) #*** define the t=0 u(r, theta,0)
                self.u_dot_present[i,j] = self.u_dot_0(r,theta) #*** define the t=0 u_dot(x,0)
                self.u_past[i,j] += self.u_present[i,j] - \
                self.u_dot_present[i,j]  * self.delta_t#*** Implement equation (B)
        self.t_now = 0.
                 
        
    def u_wave_step(self):
        """Returns the wave at the next time step, t_now + delta_t.  
        """
        # Here's where we implement the time-advancing equation for u
        u_future = np.zeros((len(self.x_pts),len(theta_pts)))  # initiate to zeros
        # also changed the initial arange values. There are more than likely issues with evaluating 1/r[i] and
        # 1/r[i]**2 at r=0
        for i in np.arange(0, len(x_pts) - 1):
            r = self.r_pts[i]
            for j in np.arrange(0, len(theta_pts) - 1):
                u_future[i,j] =2. * self.u_present[i,j] - self.u_past[i,j]\
                + (self.c*self.delta_t)**2\
                * (self.u_present[i+1,j]-2 * self.u_present[i,j] + self.u_present[i-1,j])/(self.delta_r)**2\
                + (self.c*self.delta_t)**2\
                * 1./r[i]*(self.u_present[i+1,j]-self.u_present[i-1,j])/(self.delta_r)\
                +(self.c*self.delta_t)**2\
                *1./r[i]**2*(self.u_present[i,j+1]-2 * self.u_present[i+j]+self.u_present[i,j-1])\
                /(self.delta_theta)**2            
        # update past and present u
        self.u_past = self.u_present
        self.u_present = u_future
        
        return u_future
    
    def u_wave_at_t(self, t):
        """
        Returns the wave at time t by calling u_wave_step multiple times
        """
            
        self.u_start()   # reset to the beginning for now (sets t_now=0)
        if (t < self.delta_t):
            return self.u_present
        else:        
            for step in np.arange(self.t_now, t+self.delta_t, self.delta_t):
                u_future = self.u_wave_step()
        return u_future    

In [16]:
n = 3
m = 1
a = 1
c = 1
r_min = 0
r_max = a
delta_r = 0.01
r_pts = np.arange(r_min, r_max + delta_r, delta_r)
theta_min = 0
theta_max = 2*np.pi
delta_theta = 0.01
theta_pts = np.arange(theta_min, theta_max + delta_theta, delta_theta)
delta_t = 0.01
t_min = 0
t_max = 5
t_pts = np.arange(t_min, t_max + delta_t, delta_t)

u_eigenmode = uDisk(r_pts, theta_pts, delta_t, c, a, n, m)
# At this moment, I do not know how to plot the resulting motion so that we can see the oscilalting membrane and 
# nor do I know if we have the desired result since there could be issues with 1/r & 1/r^2 in the u_wave_step routine.
# There could also be issues with the discretization of r, theta, or t. They may be too large or small and our
# numerical solution may diverge from what we want. I will definitely resubmit the project for regrading

# In the next iteration of this notebook, I want to display the resulting motion at different instances in time, 
# possibly show what happens when the discretizations are changed from the values that give us the correct
# solution, and finally, animate the correct solution.

